In [1]:
%pip install wget
%pip install gensim pyLDAvis matplotlib
%pip install pyLDAvis.gensim gensim.corpora gensim.models 

In [92]:

import pandas as pd
import numpy as np
import zipfile
import wget

In [93]:
import pyLDAvis
#import pyLDAvis.gensim #don't skip
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel


%matplotlib inline

In [94]:
train = pd.read_csv('data/train', sep='\t')

In [95]:

pd.read_csv('data/test', sep='\t')

,CLIENT_ID,RETRO_DT,tokens,urls_hashed
0,5467000,20200419,vezet 1 hats 2 сухого 1 деньги 1 режим 4 недел...,4104c0437f707b9f443bf845d92d3633 1 83e85bee0d8...
1,6093848,20201107,статью 1 month 4 udobreniya 1 podkormka 1 акту...,db0853001fff2bbe88cb5fa709d8077e 1 6e617b4bfdb...
2,5269916,20200305,zdorovye 2 мобильные 1 снять 2 tovara 1 платье...,1889e0cbd3e343e9521a07aa312257ac 2 aee71c8d18d...
3,816015821,20220115,шины 1 воздушный 1 вкусных 1 эскудо 1 сухого 1...,aee71c8d18d6b91b36bf599208a91777 7 6754d470b22...
4,223498561,20211204,купить 2 blyudo 2 юрист 2 sekretnye 2 funktsii...,1246f8d1136a114c448c1b2565e16698 2 0f7f6a4507e...
...,...,...,...,...
154799,820037931,20220408,материнские 1 hats 1 чёрный 1 деньги 14 бытово...,1889e0cbd3e343e9521a07aa312257ac 1 aee71c8d18d...
154800,820043321,20220408,кредит 8 служба 2 шаг 2 наличными 10 зубастики...,68d22a2c8d2fa49d80bbfe9edd8845f0 2 69adb4ce9c4...
154801,820058971,20220409,циан 1 novogodnie 2 результат 1 центр 1 платье...,a51fa12f9f44dde93d7e36e4ae1c4a0f 1 a18bcf54adc...
154802,820062491,20220409,макаронных 1 mny 1 снять 1 svete 1 платье 1 mn...,aee71c8d18d6b91b36bf599208a91777 122 d3862674d...


In [96]:
train_test = pd.concat([train[:6000], test[:2000]], axis=0)
train_test = train_test.drop(columns=['DEF'])
train_test

,CLIENT_ID,RETRO_DT,tokens,urls_hashed
0,5909886,20200911,форм 1 мачеха 1 работы 1 пасынка 1 горячую 1 м...,7bbcc146c0ba4df814701ff7297b2205 1 ee0d49cbc99...
1,1385448,20210519,platiza 1 мобильные 2 sergej 1 code 3 отказов ...,1889e0cbd3e343e9521a07aa312257ac 2 8253d8d3a67...
2,1866195,20210913,sims 3 кредит 1 языке 1 разлива 1 огромное 1 о...,6e617b4bfdbef85d8b09ec08ac72f0f2 1 94309f5f116...
3,1587081,20210702,code 1 историй 1 scf 1 шаг 1 деньги 3 серый 1 ...,1e833434273e04ba76cfcfb4b48ad21b 3 aee71c8d18d...
4,1400908,20210522,отказов 2 tovara 1 сергея 1 купальный 1 чёрный...,83e85bee0d81c0e9e5047e202cddf534 3 6e617b4bfdb...
...,...,...,...,...
1995,1486477,20210610,проходите 4 возвращаются 2 отказов 14 storonu ...,1889e0cbd3e343e9521a07aa312257ac 4 aee71c8d18d...
1996,1136664,20210312,кредит 2 отказов 1 forget 2 информация 1 money...,69adb4ce9c4b9a38f3ffa6adecaba2a2 2 aee71c8d18d...
1997,5612347,20200529,мобильные 2 шины 2 устройств 4 analiz 2 космет...,7f64b671da81082c4682e223303c885d 2 aee71c8d18d...
1998,1480802,20210609,клубники 1 конфетки 1 косметика 1 ptits 3 экст...,66fe0cd72631c07cb95ac1fcccd4870a 2 4e3004530a5...


In [97]:
train_test = train_test.dropna()


In [98]:
tokens = train_test.tokens

In [100]:
tokens = train_test.tokens
token_words = []

for user in tokens:
    maxima = 0
    users_data = user.split(" ")
    one_user_data = []
    #print(users_data)
    for i in range(0, len(users_data) - 1, 2):
        if (int(users_data[i+1]) > maxima):
            maxima = int(users_data[i+1])
            one_user_data = []
            one_user_data.append(users_data[i])
            #print(one_user_data)
        elif (int(users_data[i+1]) == maxima):
            one_user_data.append(users_data[i])
        else:
            pass
    token_words.append(one_user_data)
    #print(token_words)

token_words[:10]
        




[['видео', 'смотреть', 'video', 'порно', 'секс'],
 ['займы'],
 ['завтра', 'смотреть', 'телеканала', 'channel', 'программа', 'saratov'],
 ['займ'],
 ['embed'],
 ['смотреть'],
 ['embed'],
 ['бесплатно'],
 ['займы'],
 ['москве', 'работа']]

In [101]:
#нам нужна колонка со списком оставшихся слов без чисел
#CBOW is used when sg=0
w2v_tokens = Word2Vec(token_words, vector_size=100, sg=0, window=5, min_count=8)

In [102]:
def find_vec_tokens(X_train):
    vec = np.zeros(100)
    counter = -1
    try:
        for i in X_train.split():
            counter += 1
            if i in token_words and counter%2 == 0:
                #print(i)
                vec += w2v_tokens.wv.get_vector(i)
        else:
            pass
    except Exception:
        print(i)
    return vec

In [103]:
vec_tokens = train_test['tokens'].apply(find_vec_tokens)
#print(vec.head())
#title_data = pd.DataFrame.from_records(title_vec.values, index = title_vec.index)
vec_tokens_data = pd.DataFrame.from_records(vec_tokens.values, index = vec_tokens.index)
#res = pd.concat([X_train, title_data], axis = 1, sort = False)
res_tokens = pd.concat([train_test, vec_tokens_data], axis = 1, sort = False)

In [104]:
seq = np.load('data/sequences.npy', allow_pickle=True)
seq = list(seq)[:6000]+list(seq)[:2000]


In [105]:
#нам нужна колонка со списком оставшихся слов без букв
#seq_text = [sent.split() for sent in seq]
#print(matrix_text)
#CBOW is used when sg=0
w2v_hashes = Word2Vec(seq, vector_size=100, sg=0, window=5)

In [106]:
hash_words = list(w2v_hashes.wv.index_to_key)
hash_words[:10]

['dfe3caaa3affe8ad92a7603ea454bcba',
 'e1ce545b1d7e0e41aae1f790e7be26c0',
 '7a841cfb78136d5bbaef80e38e7780f5',
 '4ca77752142277b1d67a6747845a7b7f',
 'e1c8b6074849a58cf1f20a4eee8e99a9',
 '77fb4daef0b75915c1e4f0c1fdbf8136',
 '69edacc7f0fbda42a730067d6b9d76f2',
 '3e78996cc30118fdde88a31d5ea8fe84',
 '0028c3b68113a97ccf75351d44d647cd',
 '8e00e03ea21e3e99aec3a677feb03f60']

In [107]:
token_words[:10]

[['видео', 'смотреть', 'video', 'порно', 'секс'],
 ['займы'],
 ['завтра', 'смотреть', 'телеканала', 'channel', 'программа', 'saratov'],
 ['займ'],
 ['embed'],
 ['смотреть'],
 ['embed'],
 ['бесплатно'],
 ['займы'],
 ['москве', 'работа']]

In [108]:
def find_vec_hashes(X_train):
    vec = np.zeros(100)
    counter = -1
    try:
        for i in X_train.split():
            counter += 1
            if i in hash_words and counter%2 == 0:
                #print(i)
                #print(w2v_hashes.wv.get_vector(i))
                vec += w2v_hashes.wv.get_vector(i)
            else:
                pass
    except Exception:
        print(i)
    #print(vec)
    return vec

In [109]:
vec_hashes = train_test['urls_hashed'].apply(find_vec_hashes)
print(vec_hashes.head())


0    [-3.162035869434476, 5.009779177606106, 1.0982...
1    [-8.319378468440846, 13.470709016546607, 3.077...
2    [-4.803398479707539, 7.642986696213484, 1.6128...
3    [-2.3360181886237115, 4.078171670436859, 0.868...
4    [-21.861895109526813, 31.650683070532978, 8.24...
Name: urls_hashed, dtype: object


In [110]:
#title_data = pd.DataFrame.from_records(title_vec.values, index = title_vec.index)
vec_hashes_data = pd.DataFrame.from_records(vec_hashes.values, index = vec_hashes.index)
#res = pd.concat([X_train, title_data], axis = 1, sort = False)
res_hashes = pd.concat([res_tokens, vec_hashes_data], axis = 1, sort = False)

In [ ]:
res_hashes

In [ ]:
res_hashes[:20000].drop(columns=['tokens', 'urls_hashed', 'CLIENT_ID', 'RETRO_DT']).to_csv('data/df_with_seq.csv')
res_hashes

In [111]:
vec_hashes_test = test['urls_hashed'][:9000].apply(find_vec_hashes)
vec_hashes_test_data = pd.DataFrame.from_records(vec_hashes_test.values, index = vec_hashes_test.index)
#res = pd.concat([X_train, title_data], axis = 1, sort = False)
res_hashes_test = pd.concat([test[:9000], vec_hashes_test_data], axis = 1, sort = False)


UnboundLocalError: local variable 'i' referenced before assignment

In [ ]:
vec_tokens_test = train_test['tokens'].apply(find_vec_tokens)
#print(vec.head())
#title_data = pd.DataFrame.from_records(title_vec.values, index = title_vec.index)
vec_tokens_test_data = pd.DataFrame.from_records(vec_tokens_test.values, index = vec_tokens_test.index)
#res = pd.concat([X_train, title_data], axis = 1, sort = False)
res_tokens_test = pd.concat([res_hashes_test, vec_tokens_test_data], axis = 1, sort = False)